# Tests MAG with wikipedia search

## Test the API

In [2]:
from models.qa import search_on_web

qa_pairs = [
    ('Where is malaga ?', 'https://en.wikipedia.org/wiki/M%C3%A1laga'),
    ('what is the rank of india in economic growth', 'https://en.wikipedia.org/wiki/Economy_of_India'),
    ('latest series of keeping up with the kardashians', 'https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes'),
    ('where is tchernobyl', 'https://en.wikipedia.org/wiki/Chernobyl_disaster'),
    ('what is multi layer perceptron', 'https://en.wikipedia.org/wiki/Multilayer_perceptron'),
    ('why the sky is blue', 'https://en.wikipedia.org/wiki/Sky')
]

for q, url in qa_pairs:
    print(q, search_on_web(q, engine = 'bing', test_all_engines = False, site = 'en.wikipedia.org', n = 5)['urls'])

Where is malaga ? ['https://en.wikipedia.org/wiki/M%C3%A1laga', 'https://en.wikipedia.org/wiki/Malaga,_Western_Australia', 'https://en.wikipedia.org/wiki/Province_of_M%C3%A1laga', 'https://en.wikipedia.org/wiki/Malaga,_New_Jersey', 'https://en.wikipedia.org/wiki/Malaga,_Washington']
what is the rank of india in economic growth ['https://en.wikipedia.org/wiki/International_rankings_of_India', 'https://en.wikipedia.org/wiki/Economic_development_in_India', 'https://en.wikipedia.org/wiki/List_of_Indian_states_and_union_territories_by_GDP', 'https://en.wikipedia.org/wiki/Economic_history_of_India', 'https://en.wikipedia.org/wiki/List_of_countries_by_real_GDP_growth_rate']
latest series of keeping up with the kardashians ['https://en.wikipedia.org/wiki/Keeping_Up_with_the_Kardashians', 'https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes', 'https://en.wikipedia.org/wiki/Keeping_Up_with_the_Kardashians_(season_18)', 'https://en.wikipedia.org/wiki/Keeping_up_with_the

In [1]:
import json

from utils import to_json
from models.qa import answer_from_web

qa_pairs = [
    ('Where is malaga ?', 'https://en.wikipedia.org/wiki/M%C3%A1laga'),
    ('what is the rank of india in economic growth', 'https://en.wikipedia.org/wiki/Economy_of_India'),
    ('latest series of keeping up with the kardashians', 'https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes'),
    ('where is tchernobyl', 'https://en.wikipedia.org/wiki/Chernobyl_disaster'),
    ('what is multi layer perceptron', 'https://en.wikipedia.org/wiki/Multilayer_perceptron'),
    ('why the sky is blue', 'https://en.wikipedia.org/wiki/Sky')
]


for q, url in qa_pairs:
    pred = answer_from_web(
        q, site = 'en.wikipedia.org',
        engine = 'bing', test_all_engines = False,
        max_negatives = 20
    )
    print("Question : {} {}".format(q, json.dumps(to_json(pred[0]), indent = 4)))

When using token / word-level tokenizer, it can be useful to add 'detach_punctuation' in cleaners
Model restoration...
Initializing submodel : model !
Optimizer 'model_optimizer' initilized successfully !
Successfully restored model from pretrained_models/m5_nq_coqa_newsqa_mag_off_entq_ct_wt_ib_2_2_dense/saving/model.json !
Model m5_nq_coqa_newsqa_mag_off_entq_ct_wt_ib_2_2_dense initialized successfully !
Restricting max_negatives to 16 instead of 20
lengths (total : 2257 ) : [70 66 316 ... 59 77 95]  - shape : [16 425]
Question : Where is malaga ? {
    "urls": [
        "https://en.wikipedia.org/wiki/M%C3%A1laga",
        "https://en.wikipedia.org/wiki/Malaga,_Western_Australia",
        "https://en.wikipedia.org/wiki/Province_of_M%C3%A1laga",
        "https://en.wikipedia.org/wiki/Malaga,_New_Jersey",
        "https://en.wikipedia.org/wiki/Malaga,_Washington"
    ],
    "candidates": [
        {
            "text": "Washington",
            "score": -1.2202414274215698,
            

## Manual test

In [16]:
import json
import urllib
import requests

query = 'what is MPLS'

params = {
    'q' : query,
    'o' : 'json',
    'kp' : '1',
    'no_redirect' : '1',
    'no_html' : '1'
}

url = 'http://api.duckduckgo.com/?' + urllib.parse.urlencode(params)
res = requests.get(url)
print(res)
res.json()

<Response [200]>


{'Abstract': 'Multiprotocol Label Switching is a routing technique in telecommunications networks that directs data from one node to the next based on labels rather than network addresses. Whereas network addresses identify endpoints the labels identify established paths between endpoints. MPLS can encapsulate packets of various network protocols, hence the multiprotocol component of the name. MPLS supports a range of access technologies, including T1/E1, ATM, Frame Relay, and DSL.',
 'AbstractSource': 'Wikipedia',
 'AbstractText': 'Multiprotocol Label Switching is a routing technique in telecommunications networks that directs data from one node to the next based on labels rather than network addresses. Whereas network addresses identify endpoints the labels identify established paths between endpoints. MPLS can encapsulate packets of various network protocols, hence the multiprotocol component of the name. MPLS supports a range of access technologies, including T1/E1, ATM, Frame Rela

In [5]:
import os
import google
import requests
import googlesearch
import wikipedia
import pandas as pd
import tensorflow as tf

from utils import to_json
from loggers import set_level
from datasets import get_dataset
from utils.text import parse_html
from models.qa import MAG

model_name = 'm6_nq_coqa_newsqa_mag_off_entq_ct_wt_ib_2_2_mean'

qa_pairs = [
    ('Where is malaga ?', 'https://en.wikipedia.org/wiki/M%C3%A1laga'),
    ('what is the rank of india in economic growth', 'https://en.wikipedia.org/wiki/Economy_of_India'),
    ('latest series of keeping up with the kardashians', 'https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes'),
    ('where is tchernobyl', 'https://en.wikipedia.org/wiki/Chernobyl_disaster'),
    ('what is multi layer perceptron', 'https://en.wikipedia.org/wiki/Multilayer_perceptron'),
    ('why the sky is blue', 'https://en.wikipedia.org/wiki/Sky')
]

q = 'Where is malaga ?'
q = 'what is the rank of india in economic growth'
q = 'latest series of keeping up with the kardashians'
q = 'where is tchernobyl'
q = 'what is multi layer perceptron'
q = 'why the sky is blue'

print("Tensorflow version : {}".format(tf.__version__))

Tensorflow version : 2.6.2


In [6]:
model = MAG(nom = model_name)

When using token / word-level tokenizer, it can be useful to add 'detach_punctuation' in cleaners
Model restoration...
Initializing submodel : model !
Optimizer 'model_optimizer' initilized successfully !
Successfully restored model from pretrained_models/m6_nq_coqa_newsqa_mag_off_entq_ct_wt_ib_2_2_mean/saving/model.json !
Model m6_nq_coqa_newsqa_mag_off_entq_ct_wt_ib_2_2_mean initialized successfully !


In [11]:
res = googlesearch.search(q + 'site:wikipedia.org')
for i, r in enumerate(res):
    if i >= 10: break
    print(r)

https://upload.wikimedia.org/wikipedia/commons/8/85/Sky-3.jpg?sa=X&ved=2ahUKEwibi7W4jrH2AhXrvKQKHeoRBsEQ_B16BAgDEAI
https://en.wikipedia.org/wiki/Sky
https://en.wikipedia.org/wiki/Wikipedia:You_do_need_to_cite_that_the_sky_is_blue
https://en.wikipedia.org/wiki/Wikipedia:You_do_need_to_cite_that_the_sky_is_blue#Reasons
https://en.wikipedia.org/wiki/Wikipedia:You_do_need_to_cite_that_the_sky_is_blue#It's_easier_to_find_a_citation_than_to_argue_over_why_it_is_not_needed
https://en.wikipedia.org/wiki/Wikipedia:You_do_need_to_cite_that_the_sky_is_blue#The_%22obvious%22_isn't_always_obvious
https://en.wikipedia.org/wiki/Diffuse_sky_radiation
https://en.wikipedia.org/wiki/Diffuse_sky_radiation#Under_an_overcast_sky
https://en.wikipedia.org/wiki/Diffuse_sky_radiation#As_a_part_of_total_radiation
https://en.wikipedia.org/wiki/Diffuse_sky_radiation#Agriculture_and_the_eruption_of_Mt._Pinatubo


In [20]:
for i, r in enumerate(res):
    if i >= 10: break
    print(r)

https://en.wikipedia.org/wiki/Cat
https://en.wikipedia.org/wiki/List_of_cat_breeds
https://en.wikipedia.org/wiki/Domestic_short-haired_cat
https://en.wikipedia.org/wiki/Feral_cat
https://en.wikipedia.org/wiki/Farm_cat
https://simple.wikipedia.org/wiki/Cat
https://www.britannica.com/animal/cat
https://icatcare.org/advice/what-is-a-cat/
https://www.dkfindout.com/us/animals-and-nature/cats/what-is-cat/
https://www.nationalgeographic.com/animals/mammals/facts/domestic-cat


In [27]:
page = wikipedia.page('Cat - Wikipedia')
print(page.title)
print(page.summary)

Catalan Wikipedia
The Catalan Wikipedia (Catalan: Viquipèdia en català) is the Catalan-language edition of the Wikipedia free online encyclopedia. It was created on 16 March 2001, just a few minutes after the first non-English Wikipedia, the German edition. With more than 697,000 articles, it is currently the 20th-largest Wikipedia as measured by the number of articles, and the fifth-largest Wikipedia in a Romance language. In April 2016, the project had 582 active editors who made at least five edits in that month. The edition is most notable for its large number of quality articles, which illustrates the Catalan language's important online presence despite being a minority language.


In [6]:
url = 'https://en.wikipedia.org/wiki/Cat'
url = 'https://en.wikipedia.org/wiki/M%C3%A1laga'
url = 'https://en.wikipedia.org/wiki/Economy_of_India'
url = 'https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes'
url = 'https://en.wikipedia.org/wiki/Chernobyl_disaster'
url = 'https://en.wikipedia.org/wiki/Multilayer_perceptron'

url = 'https://en.wikipedia.org/wiki/Sky'

page = requests.get(url)
html = page.content.decode('utf-8')
print(page)

<Response [200]>


In [5]:
#q = 'How many paws does cat have ?'
import json

set_level('warning')

for q, url in qa_pairs:
    page = requests.get(url)
    html = page.content.decode('utf-8')

    parsed = parse_html(html)
    
    data = pd.DataFrame([{
        'question'   : q,
        'paragraphs' : [p['text'] for p in parsed],
        'titles'     : [p['title'] for p in parsed]
    }])

    pred = model.predict(data, negative_mode = 'doc', max_input_length = 512, max_negatives = 25, method = 'beam', tqdm = lambda x: x)
    print("Question : {}\nWikipedia url : {} {}".format(q, url, json.dumps(to_json(pred[0]), indent = 4)))

lengths (total : 3303 ) : [269 87 147 ... 95 138 226]  - shape : [20 425]
Question : Where is malaga ?
Wikipedia url : https://en.wikipedia.org/wiki/M%C3%A1laga {
    "candidates": [
        {
            "text": "Spain",
            "score": -0.146232008934021
        },
        {
            "text": "in Spain",
            "score": -3.524770498275757
        },
        {
            "text": "Spain.",
            "score": -4.099953651428223
        },
        {
            "text": "M\u00c3\u00a1laga",
            "score": -4.296871185302734
        },
        {
            "text": "the Costa del Sol",
            "score": -5.818670749664307
        }
    ]
}
lengths (total : 3228 ) : [137 131 161 ... 321 499 423]  - shape : [13 499]
Question : what is the rank of india in economic growth
Wikipedia url : https://en.wikipedia.org/wiki/Economy_of_India {
    "candidates": [
        {
            "text": "the top-most economy",
            "score": -0.7564894556999207
        },
        {